Goal is to translate data from Pixelav (which comes in the form of 8x32 array) into something the board can understand (string of bits 00, 01, 10, 11 that corresponds to different charge thresholds). Then want to map it into array1.
\
pixelav: 0:3 y, 0:15 x (8x32 --> 256 pixels). then each pixel goes to 2 bits which classify its charge.  
\
We realized that the current pixelav output is actually **13x21**, not **8x32**, so we are truncating values from the top and bottom rows of the original shape and padding with zeroes where we need in order to reshape the array.  
\
I'm currently using example parquet files form Eliza in order to structure this script. Thse files have 28127 clusters, each with 20 13x21 arrays (one every 200ps until 4ns).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
final_size = (8,32)
thresh0 = 400
thresh1 = 1600
thresh2 = 2400

In [3]:
def reshape_pixelav(cluster, cluster_id, time_id):
    """
    Inputs: 13x21 array from Pixelav, labelled with the time slice
    Outputs: 8x32 array with (hopefully) the same data which can be fed into Array1
    
    Truncates original rows on top and bottom and adds zeros to the extra columns on the right in order to fit 8x32 shape. 
    To avoid accidentally truncating data, we add the values of each deleted pixel to the nearest row. For example, since 
    cut off row 12, if there were any charge in that row it would be added to the same column in row 9, which is the last
    column we save. A message will appear if this happens.
    """
    # truncating and overflow
    pixelav_row = np.shape(cluster)[0]
    pixelav_col = np.shape(cluster)[1]
    chop_start = 1
    chop_stop = 9
    overflow_top = np.zeros(pixelav_col)
    overflow_bottom = np.zeros(pixelav_col)
    for col in range(pixelav_col):
        for row in range(chop_start):
            overflow_top[col] += cluster[row, col]
        for row in range(chop_stop + 1, pixelav_row):
            overflow_bottom[col] += cluster[row, col]
    cluster[chop_start, :] += overflow_top
    cluster[chop_stop, :] += overflow_bottom
    chop_cluster = cluster[chop_start:chop_stop, :]

    if np.any(overflow_top) or np.any(overflow_bottom):
        print(f"Overflow detected in cluster {cluster_id} at {time_id}ps:")
        if np.any(overflow_top):
            print(f"  Top overflow: {overflow_top}")
        if np.any(overflow_bottom):
            print(f"  Bottom overflow: {overflow_bottom}")     

    #padding
    final_input = np.zeros(final_size)
    final_input[:, :chop_cluster.shape[1]] = chop_cluster
    return final_input

In [4]:
def make_bit_string(final_input):
    """
    Inputs: Reshaped 8x32 array of Pixelav data
    Outputs: String of bits in (hopefully) correct order, ready to go into Array 1
    """
    big_row_start= 0
    pixel_data = []
    while big_row_start < 8:
        for column in reversed(range(32)):
            for mini_row in range(2):
                row = mini_row + big_row_start
                charge = final_input[row, column]
                if charge <= thresh0: 
                    label = "00"
                elif thresh0 < charge < thresh1:
                    label = "01"
                elif thresh1 < charge < thresh2:
                    label = "10"
                else:
                    label = "11"
                pixel_data.append(label)
        big_row_start += 2
    bit_string = ''.join(reversed(pixel_data))
    return bit_string

In [5]:
ar1row = 256
ar1col = 16
array1 = np.zeros((ar1row,ar1col))
len_shiftreg = (68*16)+4
len_bits = (8*32)*2
len_dnn = (256-100)*16+4*3

# fake numbers for shift register and dnn parts of the array
shiftreg = [0]*len_shiftreg
dnn = [0]*len_dnn

def make_array1(bit_string):
    i=0
    for row in range(ar1row):
        for col in reversed(range(ar1col)):
            if i < len_shiftreg:
                array1[row,col] = shiftreg[i]
            elif len_shiftreg <= i < len_shiftreg + len_bits:
                array1[row, col] = int(bit_string[i - len_shiftreg])
            elif i >= len_shiftreg+len_bits:
                array1[row, col] = dnn[i - (len_shiftreg+len_bits)]
            i += 1
    return array1

In [6]:
# using eliza's BIB parquet file as a sample pixelav output
truth = pd.read_parquet('labelsBIBtracks.parquet')
recon2d = pd.read_parquet('recon2DBIBtracks.parquet')
recon3d = pd.read_parquet('recon3DBIBtracks.parquet')
print(truth.shape, recon2d.shape, recon3d.shape)

(28127, 13) (28127, 273) (28127, 5460)


In [9]:
# i'm currently just doing a few particle clusters because there are a lot, but if you want to do all just 
# switch which "for cluster_id" line is uncommented
array1_dict = {}
#for cluster_id in range(recon3d.shape[0]):
for cluster_id in range(5, 10):
    array1_dict[cluster_id] = []
    for time_idx in range(20):
        clusterSlice = recon3d.iloc[cluster_id].to_numpy().reshape(20,13,21)[time_idx]
        time_label = 200*time_idx + 200
        final_in = reshape_pixelav(clusterSlice, cluster_id, time_label) 
        bit_string = make_bit_string(final_in)
        array = make_array1(bit_string)
        array1_dict[cluster_id].append(array)

Overflow detected in cluster 5 at 1200ps:
  Top overflow: [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 202.   0.   0.
   0.   0.   0.   0.   0.   0.   0.]
Overflow detected in cluster 5 at 1400ps:
  Top overflow: [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0. 1467.
    0.    0.    0.    0.    0.    0.    0.    0.    0.]
Overflow detected in cluster 5 at 1600ps:
  Top overflow: [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0. 1520.
    0.    0.    0.    0.    0.    0.    0.    0.    0.]
Overflow detected in cluster 5 at 1800ps:
  Top overflow: [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0. 1520.
    0.    0.    0.    0.    0.    0.    0.    0.    0.]
Overflow detected in cluster 5 at 2000ps:
  Top overflow: [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0. 1520.
    0.    0.    0.    0.    0.    0.    0.    0.    0.]
Overflow detected in cluster 5 at 2200ps:
  Top overflow: [   0.    0.    0.    0.    